# Zonally Averaged Overturning Circulation

This notebook shows a simple example of calculation the zonally averaged global meridional overturning circulation - in density space - using output from either `MOM5` or `MOM6`.

**Requirements:** The `conda/analysis3` (or later) module on ARE. I recommend an ARE session with more than 14 cores to make these computations.

**Firstly,** load in the requisite libraries:

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import pandas as pd
import intake
catalog = intake.cat.access_nri
import matplotlib.pyplot as plt
import cmocean as cm
import numpy as np
from dask.distributed import Client
import xarray as xr
import cf_xarray as cfxr

In [2]:
client = Client("tcp://10.6.48.33:8786")
client

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.04/lib/python3.9/site-packages/distributed/client.py:1401: VersionMismatchWarning: Mismatched versions found

+-------------+----------------+-----------------+-----------------+
| Package     | Client         | Scheduler       | Workers         |
+-------------+----------------+-----------------+-----------------+
| dask        | 2023.8.0       | 2023.10.0       | 2023.10.0       |
| distributed | 2023.8.0       | 2023.10.0       | 2023.10.0       |
| pandas      | 2.0.3          | 2.1.1           | 2.1.1           |
| python      | 3.9.17.final.0 | 3.10.13.final.0 | 3.10.13.final.0 |
| tornado     | 6.3.2          | 6.3.3           | 6.3.3           |
+-------------+----------------+-----------------+-----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://10.6.48.33:8786' processes=1 threads=12, memory=46.00 GiB>

**At this stage**, you need to denote whether the experiment uses output from `MOM5` or `MOM6`.

This notebook is designed to use `cf-xarray` and a dictionary of `querying.getvar` arguments to load the correct variables independent of the model. 

**Next,** choose an experiment. The dictionary below denotes which experiment we want to use for each model.

We can choose experiments at any resolution. For MOM5-based runs, they can be with or without Gent-McWilliams eddy parameterisation.

For this example, we are choosing to limit ourselves to just the last 20 years of the 0.25° control simulations. If you want to increase the resolution or integrate over a longer time you might need more resources!

In [3]:
psi_args = {
    "mom5": {
        "expt": "025deg_jra55_iaf_omip2_cycle2",
        "variable": "ty_trans_rho",
        "start_time": "2235-01-01",
        "end_time": "2255-01-01"
    },
    # "mom6": {
    #     "expt": "OM4_025.JRA_RYF",
    #     "variable": "vmo",
    #     "start_time": "1980-01-01",
    #     "end_time": "2000-01-01",
    #     "frequency": "1 monthly",
    #     "attrs": {"cell_methods": "rho2_l:sum yq:point xh:sum time: mean"},
    # }
}

MOM5: Load up `ty_trans_rho` - and sum zonally.
Also, if there is a `ty_trans_rho_gm` variable saved, assume that GM is switched on and load that as well. Most ACCESS-OM2 and MOM6 simulations save transport with units of kg/s - convert to Sv.

In [4]:
def load_streamfunction(model):
    expt = psi_args[model]["expt"]
    start_time = psi_args[model]["start_time"]
    end_time = psi_args[model]["end_time"]
    variable=psi_args[model]['variable']

    cat_subset = catalog[expt]
    var_search = cat_subset.search(variable=variable)
    darray = var_search.to_dask()
    darray = darray[variable]
    psi = darray
    psi = psi.sel(time=slice(start_time, end_time))
    psi = psi.cf.sum("longitude")

    psiGM = xr.zeros_like(psi)

    varlist = cc.querying.get_variables(session, expt)
    if varlist['name'].str.contains('ty_trans_rho_gm').any():
        GM = True
        cat_subset = catalog[expt]
        var_search = cat_subset.search(variable='ty_trans_rho_gm')
        darray = var_search.to_dask()
        darray = darray['ty_trans_rho_gm']
        psiGM = darray
        psiGM = psiGM.cf.sum("longitude")
    else:
        GM = False
    
    ρ0 = 1025 # mean density of sea-water [kg/m³]
    
    psi = psi / (1e6 * ρ0)
    psiGM = psiGM / (1e6 * ρ0)
    
    return psi, psiGM, GM

Now, we define a function that cumulatively sums the transport in the vertical. Note that in MOM5 the `ty_trans_rho_GM` variable is computed differently and **does not** require summing in the vertical. Once the calculation has been laid out, we then load the variable to force the computation to occur.

In [5]:
def sum_in_vertical(psi, psiGM, GM):
    psi_avg = psi.cf.cumsum("vertical").mean("time") - psi.cf.sum("vertical").mean("time")

    if GM:
        psi_avg = psi_avg + psiGM.cf.mean("time")

    psi_avg.load()
    
    return psi_avg

Let's load everything from a MOM5 model.

In [6]:
model = 'mom5'
psi, psiGM, GM = load_streamfunction(model)
psi_avg = sum_in_vertical(psi, psiGM, GM)

ESMDataSourceError: Failed to load dataset with key='ocean_month.1mon'
                 You can use `cat['ocean_month.1mon'].df` to inspect the assets/files for this key.
                 

Now we are ready to plot. We usually plot the streamfunction over a reduced range of density levels to highlight the deep ocean contribution...

In [ ]:
plt.figure(figsize=(10, 5))

max_psi = 25 # Sv

# how we compute the levels may seem complicated, we just want to avoid a 0 contour
# so that the plot looks soothing to the eye
levels =  np.hstack((np.arange(-max_psi, 0, 2), np.flip(-np.arange(-max_psi, 0, 2))))
cbarticks = np.hstack((np.flip(-np.arange(3, max_psi, 6)), np.arange(3, max_psi, 6)))

psi_avg.plot.contourf(levels=levels,
                      cmap=cm.cm.curl, extend='both',
                      cbar_kwargs={'shrink': 0.7, 'label': 'Sv', 'ticks': cbarticks})

psi_avg.plot.contour(levels=levels, colors='k', linewidths=0.25)

plt.gca().invert_yaxis()

plt.ylim((1037.5, 1034))
plt.ylabel('Potential Density (kg m$^{-3}$)')
plt.xlabel('Latitude ($^\circ$N)')
plt.xlim([-75, 80])
plt.title(f'Overturning in {model}: {psi_args[model]["expt"]}');

Now let's do it again from a MOM6 model. Easy!

In [ ]:
model = 'mom6'
psi, psiGM, GM = load_streamfunction(model)
psi_avg = sum_in_vertical(psi, psiGM, GM)

In [ ]:
plt.figure(figsize=(10, 5)) 

max_psi = 25 # Sv

# how we compute the levels may seem complicated, we just want to avoid a 0 contour
# so that the plot looks soothing to the eye
levels =  np.hstack((np.arange(-max_psi, 0, 2), np.flip(-np.arange(-max_psi, 0, 2))))
cbarticks = np.hstack((np.flip(-np.arange(3, max_psi, 6)), np.arange(3, max_psi, 6)))

psi_avg.plot.contourf(levels=levels,
                      cmap=cm.cm.curl, extend='both',
                      cbar_kwargs={'shrink': 0.7, 'label': 'Sv', 'ticks': cbarticks})

psi_avg.plot.contour(levels=levels, colors='k', linewidths=0.25)

plt.gca().invert_yaxis()

plt.ylim((1037.5, 1034))
plt.ylabel('Potential Density (kg m$^{-3}$)')
plt.xlabel('Latitude ($^\circ$N)')
plt.xlim([-75, 80])
plt.title(f'Overturning in {model}: {psi_args[model]["expt"]}');

Alternatively, you may want to stretch your axes to minimise the visual impact of the surface circulation, while showing the full-depth ocean.

In [ ]:
stretching_factor = 4 # A powvalues set the stretching

ρmin = psi_avg.cf['vertical'].min()

psi_avg_plot = psi_avg.assign_coords(
    {
        psi_avg.cf["vertical"].name: (psi_avg.cf["vertical"] - ρmin)**stretching_factor
    }
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

max_psi = 25 # Sv

# how we compute the levels may seem complicated, we just want to avoid a 0 contour
# so that the plot looks soothing to the eye
levels =  np.hstack((np.arange(-max_psi, 0, 2), np.flip(-np.arange(-max_psi, 0, 2))))
cbarticks = np.hstack((np.flip(-np.arange(3, max_psi, 6)), np.arange(3, max_psi, 6)))

yticks = np.array([1030, 1032, 1033, 1034, 1035, 1036, 1036.5, 1037])

psi_avg_plot.plot.contourf(levels=levels, 
                           cmap=cm.cm.curl, extend='both',
                           cbar_kwargs={'shrink': 0.7,'label': 'Sv', 'ticks': cbarticks})

psi_avg_plot.plot.contour(levels=levels, colors='k', linewidths=0.25)

plt.gca().set_yticks((yticks - ρmin.values)**stretching_factor)
plt.gca().set_yticklabels(yticks)

# ylims: a bit less than the minimnum and a bit more than the maximum values
plt.gca().set_ylim([((yticks - ρmin.values).min() / 1.1)**stretching_factor, ((yticks - ρmin.values).max() * 1.02)**stretching_factor])
plt.gca().invert_yaxis()

plt.ylabel('Potential Density (kg m$^{-3}$)')
plt.xlabel('Latitude ($^\circ$N)')
plt.xlim([-75, 80])
plt.title(f'Overturning in {model}: {psi_args[model]["expt"]}');

**Notes:**

 * We have not included the submesoscale contribution to the meridional transport in these calculations, as it tends to be relatively unimportant for the deep circulation, which we where we are primarily interested.

 * These metrics do not use mathematically correct zonal averaging in the tripole region, north of 65°N!